# Hello! Welcome to the Amazon Neptune Deep Dive Workshop!

## Building your first graph application with Amazon Neptune

In the following workshop, we will take you through the process of reviewing a graph data model, ingesting data into Neptune, and how to write graph queries using both Apache Tinkerpop Gremlin and RDF/SPARQL.  The following exercises are meant to introduce concepts related to graph databases and their specific access patterns.  We will use Jupyter Notebooks throughout this workshop.  Notebooks are great for testing code in a self-documented manner.  Feel free to add new cells to the following notebooks and write your own code, graph traversals etc.  If you have never used a Jupyter Notebook before, please refer to the 'User Interface Tour' within the Help menu above for a quick walk-through.  

These notebooks will also make use of the Neptune Workbench, a set of custom Jupyter magic commands that make it easier to interact with Neptune and its features.  For a walk-through of all of the Neptune Workbench commands, you can find sample notebooks here:

- [Getting Started with the Neptune Workbench](../../01-Getting-Started)
- [Visualizing Query Results using the Neptune Workbench](../../02-Visualization)

# SPARQL Workshop - Step 00:  Loading Data Into Neptune

Please follow the instructions to load the initial set of data which you would use for the rest of the tutorial.

## Load the Dataset

To start the workshop, we will load a sample dataset into our Neptune Cluster.  The Neptune Cluster we will be using was created for us already.  Neptune provides a [Bulk Load API](https://docs.aws.amazon.com/neptune/latest/userguide/bulk-load.html) that can load data directly from an S3 bucket.  To make this even easier, we have created a `%load` Jupyter magic and widget to construct this API call.  We will be using `%load` in this step.

In the following cells, we will issue two bulk load API calls:  one to load all the RDF Quads of our sample data set and a second to check the status of our load.  Kick off these steps by highlighting each cell and then clicking the Run button in the toolbar above (or you can press ctrl-return or ctrl-enter when a cell is highlighted).  __Be sure to wait for a cell to complete before moving to the next cell.__  Completion of cell execution will be denoted by an integer appearing within the In \[ \] to the left of each cell.  If an asterisk (\*) appears in brackets, then the cell is still running.  Wait for the asterisk to turn into a number before proceeding forward.

Go ahead and run this next cell.  This will pre-populate our `%load` widget with the necessary fields to issue our bulk load job.  Once the widget appears, click on the __SUBMIT__ button to start the load.  This should only take a couple of seconds to complete.  This will load a portion of our graph dataset that we will use for this workshop. 

In [ ]:
# Use the boto3 session SDK to fetch the region being used for this workshop
import boto3
region = boto3.Session().region_name

# Dynamically build the S3 path based on the region
s3datapath = "s3://ee-assets-prod-" + region + \
    "/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/data/imdb-rdf-social/"

# Using the Neptune Workbench's %load magic to instantiate a bulk load from the social graph in S3
%load -s {s3datapath} -f nquads -p OVERSUBSCRIBE --store-to result

Now let's check the details of our bulk load job to determine what has loaded and the source of any errors.  Run the following cell with the `%load_status` magic to see the details of our bulk load job.  

In [ ]:
# Check the status of our bulk load job with full details and any errors
%load_status {result['payload']['loadId']} --details --errors

__BEFORE MOVING FORWARD__ be sure the above command returned a payload status of `LOAD_COMPLETED: 7`.  If it returned `LOAD_NOT_STARTED` or `LOAD_IN_PROGRESS` rerun the cell block again until you see the required `COMPLETED` and `FAILED` status.

The above commands sent a request to the Neptune Cluster /loader API to load a small portion of our total dataset into Neptune from text files hosted on S3.  For RDF, Neptune supports four RDF file serialziation formats: Turtles, N-Triples, N-Quads, and RDFXML.  For this workshop, we have transformed this dataset into NQUADS format (NQ files).  Each row in the NQUADs files represents a "triple", or a unique vertex, property, or relationship (edge).  Feel free to download one of the NQUADs files and inspect it further:

##### Person Vertex File - RDF Quads describing person's in the dataset
http://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/data/imdb-rdf-social/person-nodes.nq

##### Person-to-Actor File - RDF Quads modeling relations from Person to Actor
http://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/data/imdb-rdf-social/person-artist-edges.nq

More info on the supported RDF file types for bulk load can be found here: https://docs.aws.amazon.com/neptune/latest/userguide/bulk-load-tutorial-format-rdf.html

NOTE: The remaining portion of the dataset used in this workshop has been preloaded for you to save time.

## Check Counts

Now that we've loaded the data into Neptune, let's check and make sure we have the correct counts for the two nodes types that were loaded (both the Person and Place nodes).  Assert we've loaded all the data by running the following commands:

In [ ]:
!pip install ipython_unittest
%load_ext ipython_unittest

In [ ]:
%%sparql --store-to placeCounts

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>

SELECT (COUNT(?spl) as ?numplaces)
WHERE
{
  ?spl rdf:type class:Place
}

In [ ]:
%%unittest

"Count the number of Place vertices"
assert 1460 == int(placeCounts['results']['bindings'][0]['numplaces']['value'])

In [ ]:
%%sparql --store-to personCounts

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>

SELECT (COUNT(?s) as ?numpersons)
WHERE 
{
    ?s rdf:type class:Person
}

In [ ]:
%%unittest

"Count the number of Person vertices"
assert 9892 == int(personCounts['results']['bindings'][0]['numpersons']['value'])

### Once you have completed this notebook, please move on to notebook 01.